In [3]:
import zipfile
import os

# Set paths
zip_path = "C:/Users/Ankit/OneDrive/Desktop/demo_soil/archive (5).zip"  # Change this to your actual file name
extract_path = "C:/Users/Ankit/OneDrive/Desktop/demo_soil/PlantVillage"

# Extract ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted successfully!")

✅ Dataset extracted successfully!


In [4]:
import os
import shutil

# Define paths
main_dataset_path = "C:/Users/Ankit/OneDrive/Desktop/demo_soil/PlantVillage"
folder1 = os.path.join(main_dataset_path, "PlantVillage")
folder2 = os.path.join(main_dataset_path, "plantvillage")
final_dataset_path = "C:/Users/Ankit/OneDrive/Desktop/demo_soil/Final_PlantVillage"

# Ensure the final dataset path exists
os.makedirs(final_dataset_path, exist_ok=True)

# Move class folders from both subdirectories to the final dataset
for folder in [folder1, folder2]:
    if os.path.exists(folder):
        for subfolder in os.listdir(folder):
            src = os.path.join(folder, subfolder)
            dst = os.path.join(final_dataset_path, subfolder)
            if os.path.isdir(src):
              
                shutil.move(src, dst)  # Move the class folder

print("✅ Merged all class folders into:", final_dataset_path)


✅ Merged all class folders into: C:/Users/Ankit/OneDrive/Desktop/demo_soil/Final_PlantVillage


In [5]:
import os

# List classes
print("Classes:", os.listdir(final_dataset_path))


Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'PlantVillage', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_healthy', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato__Tomato_YellowLeaf__Curl_Virus']


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# 🔹 Set your dataset path (Update this based on your system)
dataset_path = final_dataset_path  # Update this!

# 🔹 Check if the dataset path exists
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset not found at {dataset_path}. Please check the path.")

# 🔹 Data Preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    dataset_path, target_size=(128, 128), batch_size=32, subset="training"
)

val_data = datagen.flow_from_directory(
    dataset_path, target_size=(128, 128), batch_size=32, subset="validation"
)

# 🔹 CNN Model Definition
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_data.class_indices), activation='softmax')  # Multi-class probability output
])

# 🔹 Compile & Train the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🟢 Training Started...")
model.fit(train_data, validation_data=val_data, epochs=5)

# 🔹 Save the Model
model.save("plant_disease_model.h5")
print("\n✅ Model Saved as 'plant_disease_model.h5'")

# 🔹 Mapping class indices to labels
class_labels = {v: k for k, v in train_data.class_indices.items()}

# 🔹 Function to Predict Disease Probabilities from an Image
def predict_plant_disease(image_path, model_path="plant_disease_model.h5"):
    model = load_model(model_path)  # Load trained model

    # Load image & preprocess
    img = image.load_img(image_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Reshape for model input

    # Predict
    predictions = model.predict(img_array)[0]  # Get probability scores
    probabilities = {class_labels[i]: round(float(prob), 4) for i, prob in enumerate(predictions)}

    # Sort by highest probability
    sorted_probs = dict(sorted(probabilities.items(), key=lambda item: item[1], reverse=True))

    print("\n🔍 Prediction Probabilities:")
    for disease, prob in sorted_probs.items():
        print(f"{disease}: {prob * 100:.2f}%")

    return sorted_probs

Found 66048 images belonging to 16 classes.
Found 16504 images belonging to 16 classes.


e:\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



🟢 Training Started...


e:\Anaconda\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 923s 435ms/step - accuracy: 0.4970 - loss: 1.4484 - val_accuracy: 0.3351 - val_loss: 2.0797
Epoch 2/5
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 1306s 633ms/step - accuracy: 0.5304 - loss: 0.8919 - val_accuracy: 0.2766 - val_loss: 2.5978
Epoch 3/5
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 874s 423ms/step - accuracy: 0.5441 - loss: 0.8103 - val_accuracy: 0.2734 - val_loss: 2.9538
Epoch 4/5
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 893s 432ms/step - accuracy: 0.5456 - loss: 0.7698 - val_accuracy: 0.2163 - val_loss: 3.4111
Epoch 5/5
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 853s 413ms/step - accuracy: 0.5499 - loss: 0.7436 - val_accuracy: 0.2401 - val_loss: 3.5738



✅ Model Saved as 'plant_disease_model.h5'


In [9]:
# 🔹 Example Usage
test_image_path = "C:/Users/Ankit/OneDrive/Desktop/demo_soil/tomato leaf mold.jpg"
if os.path.exists(test_image_path):
    predict_plant_disease(test_image_path)
else:
    print("\n⚠️ Test image not found. Please provide a valid path.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

🔍 Prediction Probabilities:
PlantVillage: 56.35%
Pepper__bell___healthy: 23.22%
Tomato_Late_blight: 19.76%
Pepper__bell___Bacterial_spot: 0.53%
Tomato__Tomato_YellowLeaf__Curl_Virus: 0.07%
Tomato_Septoria_leaf_spot: 0.05%
Tomato_Early_blight: 0.02%
Potato___Early_blight: 0.00%
Potato___Late_blight: 0.00%
Potato___healthy: 0.00%
Tomato_Bacterial_spot: 0.00%
Tomato_Leaf_Mold: 0.00%
Tomato_Spider_mites_Two_spotted_spider_mite: 0.00%
Tomato__Target_Spot: 0.00%
Tomato__Tomato_mosaic_virus: 0.00%
Tomato_healthy: 0.00%
